In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import tensorflow as tf
import wandb
import pandas as pd
from prosit_t.wandb_agent.train_utils import get_proteometools_data
from dlomix.losses import masked_spectral_distance
import os
from prosit_t.models import PrositTransformerV2
from dlomix.models import PrositIntensityPredictor
import numpy as np
import tensorflow.keras.backend as K
from prosit_t.inference.visualization import compare_spectra, compare_multiple_spectra

2023-10-02 15:16:17.327120: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-02 15:16:57.054652: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-packages/wandb/integration/keras/keras.py", line 86, in patch_tf_keras
    from keras.engine import training
ModuleNotFoundError: No module named 'keras.engine'
wandb: ERROR Unable to patch Tensorflow/Keras
exception while trying to patch_tf_keras
Traceback (most recent call last):
  File "/cmnfs/home/l.mamisashvili/miniconda3/envs/prosit-t/lib/python3.9/site-p

In [4]:
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
physical_devices = tf.config.list_physical_devices("GPU")
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [5]:
data_config = {
    "dataset": "proteometools",
    "data_source": {
        "train": "/cmnfs/proj/prosit/Transformer/first_pool_train.parquet",
        "val": "/cmnfs/proj/prosit/Transformer/first_pool_test.parquet",
    },
    "fragmentation": "HCD",
    "batch_size": 1024,
    "seq_length": 30,
}

In [6]:
project_name = "transforming-prosit-first-pool"
run = wandb.init(project=project_name)

wandb: Currently logged in as: mamisashvili-lizi (prosit-compms). Use `wandb login --relogin` to force relogin


In [7]:
_, val_data = get_proteometools_data(data_config)

2023-10-02 15:23:07.795144: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 43640 MB memory:  -> device: 0, name: NVIDIA A40, pci bus id: 0000:e2:00.0, compute capability: 8.6


In [8]:
batch_idx = 0
sample_idx = 456
batch_x, batch_y = [x for x in val_data.skip(batch_idx).take(1)][0]

In [9]:
batch_x

{'sequence': <tf.Tensor: shape=(1024, 30), dtype=string, numpy=
 array([[b'T', b'A', b'A', ..., b'', b'', b''],
        [b'W', b'I', b'I', ..., b'', b'', b''],
        [b'F', b'E', b'L', ..., b'', b'', b''],
        ...,
        [b'Q', b'L', b'E', ..., b'', b'', b''],
        [b'A', b'I', b'E', ..., b'', b'', b''],
        [b'A', b'I', b'G', ..., b'', b'', b'']], dtype=object)>,
 'collision_energy': <tf.Tensor: shape=(1024, 1), dtype=float64, numpy=
 array([[0.21534173],
        [0.21220896],
        [0.24663639],
        ...,
        [0.32626746],
        [0.21299643],
        [0.24857905]])>,
 'precursor_charge': <tf.Tensor: shape=(1024, 6), dtype=float32, numpy=
 array([[0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0., 0.],
        ...,
        [0., 1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0.],
        [0., 1., 0., 0., 0., 0.]], dtype=float32)>}

In [19]:
arr = np.array(test_seq)
len(arr[arr != b''])

18

In [21]:
test_seq = [b'V', b'R', b'E', b'E', b'V', b'P', b'L', b'E', b'L', b'V', b'E',
        b'A', b'H', b'V', b'K', b'', b'', b'', b'', b'', b'', b'', b'',
        b'', b'', b'', b'', b'', b'', b'']
for batch_idx, batch in enumerate(val_data):
    x, _ = batch

    mask = (
        tf.math.less_equal(
            abs(tf.squeeze(x["collision_energy"]) - 0.3144403), 1e-5
        )
        & tf.reduce_all(
            tf.equal(x["precursor_charge"], [0.0, 0.0, 1.0, 0.0, 0.0, 0.0]), axis=1
        )
        & tf.reduce_all(tf.equal(x["sequence"], test_seq), axis=1)
    )

    indices = tf.where(mask)
    if len(indices) > 0:
        print("index in batch", indices, "batch index", batch_idx)
        print(x["collision_energy"][int(tf.squeeze(indices))])
        print(x["precursor_charge"][int(tf.squeeze(indices))])
        print(x["sequence"][int(tf.squeeze(indices))])

index in batch tf.Tensor([[7]], shape=(1, 1), dtype=int64) batch index 121
tf.Tensor([0.3144403], shape=(1,), dtype=float64)
tf.Tensor([0. 0. 1. 0. 0. 0.], shape=(6,), dtype=float32)
tf.Tensor(
[b'V' b'R' b'E' b'E' b'V' b'P' b'L' b'E' b'L' b'V' b'E' b'A' b'H' b'V'
 b'K' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''], shape=(30,), dtype=string)
index in batch tf.Tensor([[1004]], shape=(1, 1), dtype=int64) batch index 600
tf.Tensor([0.3144403], shape=(1,), dtype=float64)
tf.Tensor([0. 0. 1. 0. 0. 0.], shape=(6,), dtype=float32)
tf.Tensor(
[b'V' b'R' b'E' b'E' b'V' b'P' b'L' b'E' b'L' b'V' b'E' b'A' b'H' b'V'
 b'K' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b'' b''], shape=(30,), dtype=string)
